In [4]:
# pip install py_vncorenlp

In [34]:
import pandas as pd
import numpy as np
import torch
from pyvi import ViTokenizer

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import numpy
import re
# import underthesea # Tokenizeer

from sklearn.model_selection import train_test_split 

from transformers import AutoModel, AutoTokenizer # BERT

# SVM
from sklearn.svm import SVC
from joblib import dump

In [40]:
# check data

train_df = pd.read_csv('./data/train.csv', names=['label', 'content'])
train_df.head(2)

,label,content
0,0,máy dùng hay bị đơ máy
1,0,chỉ có dây cáp nguồn không có adapter sao sử d...


In [50]:
# Hàm load model BERT
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

In [51]:
def standardize_data(row):
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row

In [52]:
def load_stopwords():
    sw = []
    with open("vietnamese-stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        sw.append(line.replace("\n",""))
    return sw

In [53]:
def load_data():
    v_text = []
    v_label = []

    df = pd.read_csv('./data/train.csv', names=['label', 'content'], encoding='utf-8')

    for _, row in df.iterrows():
        content = row['content']
        label = row['label']
        
        v_text.append(standardize_data(content))
        v_label.append(int(label))

    print(v_label)
    return v_text, v_label

In [59]:
def make_bert_features(v_text):
    global phobert, sw
    v_tokenized = []
    max_len = 100  # Mỗi câu dài tối đa 100 từ

    for i_text in v_text:
        print("Đang xử lý line = ", i_text)
        # Tokenize bằng ViTokenizer
        line = ViTokenizer.tokenize(i_text)
        
        # Lọc stopwords
        filtered_sentence = [w for w in line.split() if not w in sw]
        
        # Ghép lại thành câu sau khi lọc
        line = " ".join(filtered_sentence)
        
        # Tokenize bằng BERT tokenizer
        line = tokenizer.encode(line, add_special_tokens=True)  # Thêm các token đặc biệt cho BERT
        
        v_tokenized.append(line)

    # Padding các câu để có độ dài đồng nhất
    padded = np.array([i + [1] * (max_len - len(i)) if len(i) < max_len else i[:max_len] for i in v_tokenized])
    print('padded:', padded[0])
    print('len padded:', padded.shape)

    # Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
    attention_mask = np.where(padded == 1, 0, 1)
    print('attention mask:', attention_mask[0])

    # Chuyển thành tensor
    padded = torch.tensor(padded).to(torch.long)
    print("Padd = ", padded.size())
    attention_mask = torch.tensor(attention_mask)

    # Lấy features đầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = phobert(input_ids=padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()  # Lấy đặc trưng từ token [CLS]
    print(v_features.shape)

    return v_features


In [61]:
print("Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...")
sw = load_stopwords()
print("Đã nạp xong danh sách các từ vô nghĩa")

print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
print("Đã nạp xong model BERT.")

print("Chuẩn bị load dữ liệu....")
text, label = load_data()
print("Đã load dữ liệu xong")

print("Chuẩn bị tạo features từ BERT.....")
features = make_bert_features(text)
print("Đã tạo xong features từ BERT")

Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...
Đã nạp xong danh sách các từ vô nghĩa
Chuẩn bị nạp model BERT....
Đã nạp xong model BERT.
Chuẩn bị load dữ liệu....
[0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 2, 0, 1, 2, 2, 0, 0, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 1, 1, 0, 1, 2, 2, 2, 2, 0, 0, 0, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 2, 0, 0, 2, 1, 2, 0, 1, 2, 1, 1, 2, 0, 0, 1, 2, 1, 2, 0, 2, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 0, 2, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 2, 2, 0, 1, 0, 0, 2, 1, 2, 2, 1, 0, 0, 1, 1, 1, 2, 1, 0, 2, 0, 0, 2, 0, 1, 1, 0, 2, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 2, 2, 0, 1, 1, 0, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 1, 0, 2, 2, 1, 2, 1, 0, 0, 1, 1, 0, 0, 0, 2, 1, 1, 0, 2, 2, 1, 0, 0, 2, 0, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 0, 1, 1, 2, 2, 2, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 2, 1, 0, 2

In [62]:
# Split train, test
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=0)

In [63]:
parameters = {'kernel': ('linear', 'rbf'), 'C': [1, 2, 4], 'gamma': [0.125, 0.25, 0.5, 1, 2, 4]}
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(), param_grid=parameters)
grid_search = clf.fit(X_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print(grid_search.best_estimator_)

# best prarams
print('best prarams:', clf.best_params_)

Best score: 0.604
SVC(C=1, gamma=0.125, kernel='linear')
best prarams: {'C': 1, 'gamma': 0.125, 'kernel': 'linear'}


In [64]:
print("Chuẩn bị train model SVM....")
cl = SVC(kernel='linear', probability=True, gamma=0.125)
cl.fit(features, label)

Chuẩn bị train model SVM....


SVC(gamma=0.125, kernel='linear', probability=True)

In [85]:
sc = cl.score(X_test, y_test)
print('Accuracy: ', sc*100, '%')

Accuracy:  91.11842105263158 %


In [66]:
dump(cl, 'save_model.pkl')
print("Đã lưu model SVM vào file save_model.pkl")

Đã lưu model SVM vào file save_model.pkl


In [67]:
from joblib import load

model = load('save_model.pkl')

In [84]:
new_texts = ["Sản phẩm bị hư trong vòng 2 tuần sử dụng, dịch vụ hỗ trợ không hiệu quả.", 
            "Sản phẩm cực tốt, âm thanh nghe rất chân thật, tôi rất thích", 
            "Sản phẩm này nhìn chung ổn trong tầm giá, tính năng không có gì nổi trội, xài tạm thời thì ok"]
standardized_texts = [standardize_data(text) for text in new_texts]
features_new = make_bert_features(standardized_texts)

predictions = model.predict(features_new)
print("Kết quả phân loại:", predictions)

Đang xử lý line =  sản phẩm bị hư trong vòng 2 tuần sử dụng  dịch vụ hỗ trợ không hiệu quả
Đang xử lý line =  sản phẩm cực tốt  âm thanh nghe rất chân thật  tôi rất thích
Đang xử lý line =  sản phẩm này nhìn chung ổn trong tầm giá  tính năng không có gì nổi trội  xài tạm thời thì ok
padded: [   0  265 6007  323   76  386  117  399  291  371    2    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1]
len padded: (3, 100)
attention mask: [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0